## Introduction

This notebook tries to explore the dual concepts of data and recursion, and how they can be applied to common programming tasks.

**Data** represents finite values. A given data $x$ of type $T$ is made by a finite application of _constructors_ of $T$. For eg, any finite stack is made by creating a new stack and then pushing a finite amount of values.

**Codata** (usually) represents infinite values. A given codata is defined by specifying a set of _destructor_ functions. A destructor is a function able to extract some part of the infinite structure (like `head` and `tail`).

## Recursion and Corecursion

> In computer science, corecursion is a type of operation that is dual to recursion. Whereas recursion works analytically, starting on data further from a base case and breaking it down into smaller data and repeating until one reaches a base case, corecursion works synthetically, starting from a base case and building it up, iteratively producing data further removed from a base case. [wikipedia](https://en.wikipedia.org/wiki/Corecursion)

**Recursion** can be used to define functions that map values from data $x$ by invoking itself over the parts of $x$. These functions will eventually stop recursing when reaching the base cases.

**Corecursion** defines functions that map values from _codata_ by applying destructors to the results.

Consider a recursive function to add one to every element of a list of numbers:

In [ ]:
def add(ns):
  if ns==[]:
    return []
  n, *ns = ns
  return [n+1] + add(ns)

assert add([1,2,3]) == [2,3,4]

> Corecursion is often used in conjunction with lazy evaluation, to produce only a finite subset of a potentially infinite structure (rather than trying to produce an entire infinite structure at once)

In Python we use generators to achieve lazy evaluation:

In [ ]:
# codata, list of all naturals
def nats(i=1):
  yield i
  yield from nats(i+1)

Function `head` shows the first $n$ elements of a given list or codata,

In [ ]:
def head(xs, n):
  """ generates the first n items """
  for i,x in enumerate(xs):
    if i==n:
      break
    yield x

print(*head(nats(), 30))

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30


Let's now make a corecursive function to add one to all naturals:

In [ ]:
def coadd(ns):
  yield next(ns)+1
  yield from coadd(ns)

print(*head(coadd(nats()), 30))

2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31


Another codata eg, an infinite list of ones:

In [ ]:
# In Haskell: ones = 1 : ones
def ones():
  yield 1
  yield from ones()

In [ ]:
print(*head(ones(), 10))

1 1 1 1 1 1 1 1 1 1


Corecursion does not need always to create infinite codata. The next corecursive function produces a finite list when $a\leq b$,

In [ ]:
def count_upto(a, b):
  if a != b+1:
    yield a
    yield from count_upto(a+1, b)

In [ ]:
print(*head(count_upto(2, 9), 30))
print(*head(count_upto(9, 2), 30))

2 3 4 5 6 7 8 9
9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38


Corecursion creates a _stream_ of values, starting from the bases of recursion to more complex subproblems.

In [ ]:
def facts(i=0, fac=1):
  yield fac
  yield from facts(i+1, fac*(i+1))

In [ ]:
print(*head(facts(), 15))

1 1 2 6 24 120 720 5040 40320 362880 3628800 39916800 479001600 6227020800 87178291200


To make a corecursive generator, we can do without recursion,

In [ ]:
# alternatives to previous implementations
def nats():
  i = 1
  while True:
    yield i
    i += 1

def coadd(ns):
  for n in ns:
    yield n+1

Let's check some more non-recursive examples.

Returns the odd-indexed elements:

In [ ]:
def odds(xs):
  while True:
    yield next(xs)
    next(xs)

print(*head(odds(facts()), 10))

1 2 24 720 40320 3628800 479001600 87178291200 20922789888000 6402373705728000


Python iterable functions work well in this context:

In [ ]:
# remove all factorials having one or more digits 3
not3 = lambda n: '3' not in str(n)

print(*head(filter(not3, facts()), 10))

1 1 2 6 24 120 720 5040 479001600 6227020800


A stream of all factorials,

In [ ]:
def facts():
  i, fac = 0, 1
  while True:
    yield fac
    i, fac = i+1, fac*(i+1)

The Fibonacci sequence,

In [ ]:
def fibs():
  a, b = 0, 1
  while True:
    yield a
    a, b = b, a + b

In [ ]:
print(*head(fibs(), 25))

0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 6765 10946 17711 28657 46368


A corecursive generator for $\mathbb{Q}$,

In [ ]:
def rationals():
  yield (1,1)
  a = [1,1]
  k = 1
  while True:
    if k % 2 == 0:
      a.append( a[k//2] )
    else:
      kHalf = k//2
      a.append (a[kHalf]+a[kHalf+1] )
    yield (a[k], a[k+1])
    k = k+1

In [ ]:
format = lambda q: f'{q[0]}/{q[1]}'

print(*map(format, head(rationals(), 20)))

1/1 1/2 2/1 1/3 3/2 2/3 3/1 1/4 4/3 3/5 5/2 2/5 5/3 3/4 4/1 1/5 5/4 4/7 7/3 3/8


The corecursive version of the sieve of Eratosthenes:

In [ ]:
from itertools import count

def sieve(ns):
  n = next(ns)
  yield n
  yield from sieve( i for i in ns if i%n!=0 )

def primes():
  yield from sieve(count(start=2))

In [ ]:
print(*head(primes(), 50))

2 3 5 7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89 97 101 103 107 109 113 127 131 137 139 149 151 157 163 167 173 179 181 191 193 197 199 211 223 227 229


The next example produces a stream with the [Thue-Morse sequence](https://en.wikipedia.org/wiki/Thue%E2%80%93Morse_sequence):

In [ ]:
def invert(bits):
  return ''.join('0' if bit=='1' else '1' for bit in bits)

def thue_morse():
  yield '0'
  yield from map(lambda s: s+invert(s), thue_morse())

In [ ]:
print(*head(thue_morse(), 5), sep='\n')

0
01
0110
01101001
0110100110010110


Streams can also be defined corecursively.

Consider the following corecursive Haskell definition for the Fibonacci sequence,

    fibs = 0 : 1 : zipWith (+) fibs (tail fibs)

Let's translate that to Python,    

In [ ]:
def zipWith(f, *streams):
  return (f(*xs) for xs in zip(*streams))

# or simply
zipWith = map

In [ ]:
def tail(xs):
  next(xs)
  yield from xs

In [ ]:
def fibs():
  yield 0
  yield 1
  yield from zipWith(int.__add__, fibs(), tail(fibs()))

However, this straightforward implementation is slow, since there is no memoization being done (both streams are independent):

In [ ]:
print(*head(fibs(), 26))

0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 6765 10946 17711 28657 46368 75025


We can use function `tee` to provides several iterators to the same stream:

<!-- old version

from itertools import tee, islice, chain
from operator import add

def fibs():
  def output():
    for i in chain((0,1), map(add, fib, tail)):
      yield i

  stream, fib, tail = tee(output(), 3)
  tail = islice(tail, 1, None)
  return stream

-->

In [ ]:
from itertools import tee

def fibs():
  yield 0
  yield 1
  s1, s2 = tee(fibs(), 2)
  yield from zipWith(int.__add__, s1, tail(s2))

In [ ]:
print(*head(fibs(), 36))

0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 6765 10946 17711 28657 46368 75025 121393 196418 317811 514229 832040 1346269 2178309 3524578 5702887 9227465


Corecursivity can be applied to other codatatypes. The next generator traverses an infinite binary tree using breath first:

In [ ]:
from queue import deque

class Tree:
  def __init__(self, val, left, right):
    self.val   = val
    self.left  = left
    self.right = right

def bf(tree):
  nodes = deque([tree])
  while nodes:
    t = nodes.popleft()
    if t is not None:
      yield t.val
      nodes.append(t.left)
      nodes.append(t.right)

In [ ]:
t1 = Tree(1, None, None)
t2 = Tree(2, None, None)
t3 = Tree(3, t1, t2)
t1.left  = t3 # make an infinite tree
t1.right = t3

print(*head(bf(t3), 50))

3 1 2 3 3 1 2 1 2 3 3 3 3 1 2 1 2 1 2 1 2 3 3 3 3 3 3 3 3 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 3 3 3 3 3


### Unfolds

A **fold** is a recursive function that analyses a data structure and recombines their elements (usually by summarizing them).

Python provides a folding function denoted `reduce`,

In [ ]:
from functools import reduce

xs = [1,2,3,4]
reduce(lambda acc,x: 2*x+acc, xs, 0)

20

An **unfold** is a corecursive function that generates a sequence by repeated application of a given function to its previous result.

A famous unfold is `iterate`:

In [ ]:
def iterate(f, x):
  yield x
  yield from iterate(f, f(x))

# alternative
def iterate(f, x):
  while True:
    yield x
    x = f(x)

Making the powers of 2 using `iterate`:

In [ ]:
print(*head(iterate(lambda x:2*x, 1), 20))

1 2 4 8 16 32 64 128 256 512 1024 2048 4096 8192 16384 32768 65536 131072 262144 524288


Let's make a more general unfold factory:

In [ ]:
def unfold(step, end, state):
  while True:
    if end(state):
      return
    value, state = step(state)
    yield value

forever = lambda _: False

Some use examples:

In [ ]:
powers2 = unfold(lambda st: (st,2*st), forever, 1)
print(*head(powers2, 20))

1 2 4 8 16 32 64 128 256 512 1024 2048 4096 8192 16384 32768 65536 131072 262144 524288


Defining `iterate` via `unfold`,

In [ ]:
iterate = lambda f, n: unfold(lambda st: (st,f(st)), forever, n)

powers3 = iterate(lambda x:3*x, 1)
print(*head(powers3, 20))

1 3 9 27 81 243 729 2187 6561 19683 59049 177147 531441 1594323 4782969 14348907 43046721 129140163 387420489 1162261467


This next function is a zip for two indexed structures:

In [ ]:
end  = lambda ps: not ps[0] or not ps[1]
step = lambda ps: ((ps[0][0],ps[1][0]), (ps[0][1:],ps[1][1:]))
zip2 = lambda xs, ys: unfold(step, end, (xs,ys))

print(*zip2('abcde',[1,2,3,4,5,6]))

('a', 1) ('b', 2) ('c', 3) ('d', 4) ('e', 5)


### Processes

This section is based on Doets and Eijck's _The Haskell Road to Logic, Math and Programming_ [[book](https://fldit-www.cs.tu-dortmund.de/~peter/PS07/HR.pdf)]

The next generator provides a stream of random integers, from zero to a given bound:

In [ ]:
from random import randint

def rnd_ints(bound):
  while True:
    yield randint(0, bound)

In [ ]:
print(*head(rnd_ints(1), 40))
print(*head(rnd_ints(4), 40))

1 1 1 0 0 0 0 1 1 0 1 1 0 1 0 1 0 1 1 0 1 0 1 0 1 0 1 1 0 1 1 1 0 1 0 0 1 0 1 1
3 1 1 1 0 3 1 4 2 3 3 4 1 1 0 4 4 3 3 4 3 1 4 0 3 1 0 4 4 3 0 3 0 1 3 3 3 2 1 3


A process, herein, is a function mapping streams of integers (that represent some type of event) to streams of strings (that describe some action performed by the physical system in response to the current event).

Processes can be used to model many different systems that need to interact with events, like clocks, traffic lights, vending machines, etc. These systems usually have different states that might react differently to the same event. This is the domain of _finite state machines_.

Function `start` initializes the given process by providing a stream of events to react to:

In [ ]:
def start(process, bound):
  return process(rnd_ints(bound))

Here's a first example. A clock that simply ticks and tacks. The events are always the same, and are used to jump between the clock's two states:

In [ ]:
def clock(events):
  next(events) # consume an event
  yield 'tick'
  yield from clock_tack(events)

def clock_tack(events):
  next(events) # consume an event
  yield 'tack'
  yield from clock(events)

In [ ]:
p = start(clock, 0)

print(*head(p, 20))

tick tack tick tack tick tack tick tack tick tack tick tack tick tack tick tack tick tack tick tack


The next process is a clicking clock but with a probability of cracking. The events here are used to define a non-deterministic system:

In [ ]:
def clock(events):
  if next(events) > 0:
    yield 'tick'
    yield from clock(events)
  else:
    yield from crack(events)

def crack(events):
  yield 'crack' # and stops the process

In [ ]:
p = start(clock, 4) # 20% probability of cracking (event 0 means crack)

print(*head(p, 20))

tick tick tick tick tick tick tick tick tick tick crack


> _Consider a very simple vending machine that sells mineral water
and beer. Water costs one euro, beer two euros. The machine has a coin slot and
a button. It only accepts 1 euro coins. If a coin is inserted and the dispense button
is pushed, it dispenses a can of mineral water. If instead of pushing the dispense
button, another one euro coin is inserted and next the dispense button is pushed,
it dispenses a can of beer. If, instead of pushing the button for beer, a third coin
is inserted, the machine returns the inserted money (three 1 euro coins) and goes
back to its initial state._ <font size="-1">from [Doets & Eijck](https://fldit-www.cs.tu-dortmund.de/~peter/PS07/HR.pdf)</font>

Each state is represent by a different process, that decides based on the current event, which will be the next state.

In [ ]:
# event 0 is inserting coin; event 1 is press release button
def vending(events):
  if next(events) == 0:
    yield 'coin'
    yield from state1(events)
  else:
    yield from vending(events) # no coins inserted

def state1(events):
  if next(events) == 0:
    yield 'coin'
    yield from state2(events)
  else:
    yield 'water'
    yield from vending(events)

def state2(events):
  if next(events) == 0:
    yield 'coin'
    yield 'money-back'
    yield from vending(events)
  else:
    yield 'beer'
    yield from vending(events)

Let's process the vending machine with some random events:

In [ ]:
p = start(vending, 1)

print(*head(p, 15))

coin coin coin money-back coin water coin water coin water coin coin coin money-back coin


## Trees as codata

A board game, like Tic-Tac-Toe, can be represented as a tree, where nodes are valid positions and, given each node, its children represent all valid positions achievable by playing one of the available legal moves.

Here's a snapshot of the first nodes of Tic-Tac-Toe game tree,

<center><img src='https://raw.githubusercontent.com/jpneto/Prog.I/master/imgs/tree_tictactoe_full.png' width=700px></center>

Consider a tree as a node with a state (the necessary information to define a valid game position), and a list of nodes representing the next valid positions (its children),

In [ ]:
class Tree:
  def __init__(self, state, children):
    self.state    = state
    self.children = children # list of Trees

  def __iter__(self):
    yield from tree_gen(self)

  def size(self, t):
    if t is None:
      return 0
    return 1 + sum(child.size() for child in t.children)

  def __repr__(self):
    return self._make_repr(0)

  def _make_repr(self, level=0):
    ret = ["   "*level + repr(self.state)]
    for child in self.children:
      ret.append(child._make_repr(level+1))
    return '\n'.join(ret)

The class is iterable, using breadth-first,

In [ ]:
from queue import Queue

def tree_gen(t):
  q = Queue() # breadth-first
  q.put(t)
  while not q.empty():
    t = q.get()
    yield t.state
    for child in t.children:
      q.put(child)

To build the game tree we could implement a recursive funciton that would go thru all computed positions to compute the next level, and on...

Another way is to define the game tree in a corecursive way.

Assume we have function `next_states :: state -> [state]` that, given a state, returns all next valid states. Given an initial state (in Tic-Tac-Toe, the empty board), a game tree is defined by the next corecursive function,

In [ ]:
def iterate_tree(next_states):
  return lambda state: Tree(state,
                            map(iterate_tree(next_states), next_states(state)))

Notice all the game tree is lazely defined. The game tree might be huge, or even infinite (for some other games), but its lazy nature allows us to compute only parts, if we wish so!

Usually we only require to compute a finite depth of the game tree. That's the task of `prune`,

In [ ]:
def prune(level):
  def prunner(tree):
    if level==0:
      return Tree(tree.state, [])
    # list computes the concrete tree, instead of each children being an iterable
    return Tree(tree.state, list(map(prune(level-1), tree.children)))
  return prunner

### Tic-Tac-Toe

Let's code the game rules of Tic-Tac-Toe,

In [ ]:
class TTT:
  """ a tictactoe state """
  def __init__(self, board, next_player):
    self.board       = board
    self.value       = 0
    self.next_player = next_player

  def __repr__(self):
    return (''.join(self.board[ :3]) + '/' +
            ''.join(self.board[3:6]) + '/' +
            ''.join(self.board[6: ]) +
            ' [' + self.next_player + ']' +
            ' ' + str(self.value))

The next predicate checks if the state is an endgame.

In [ ]:
def has_win(board, player):
  for i in range(3):
    if board[3*i:3*i+3] == [player]*3:            # check row
      return True
    if board[i::3]      == [player]*3:            # check column
      return True
  if board[0] == board[4] == board[8] == player:  # check main diagonal
    return True
  if board[2] == board[4] == board[6] == player:  # check anti diagonal
    return True
  return False

assert has_win(list('XXX......'), 'X') # rows
assert has_win(list('...OOO...'), 'O')
assert has_win(list('......XXX'), 'X')
assert has_win(list('X..X..X..'), 'X') # columns
assert has_win(list('.X..X..X.'), 'X')
assert has_win(list('..O..O..O'), 'O')
assert has_win(list('X...X...X'), 'X') # diagonals
assert has_win(list('..O.O.O..'), 'O')

assert not has_win(list('XXOOOXXXO'), 'X')

The `next_states` for Tic-Tac-Toe:

In [ ]:
def next_ttt_states(ttt_game):
  board, player = ttt_game.board, ttt_game.next_player

  new_states = []
  next_player = 'X' if player == 'O' else 'O'
  if not has_win(board, next_player): # there's only states if game has not ended
    for i in range(9):
      if board[i] == '.':
        new_board = board[:]
        new_board[i] = player
        new_states.append(TTT(new_board, next_player))
  return new_states

Let's define the game tree (define, not compute it yet),

In [ ]:
init_state = TTT(list('.'*9), 'X') # empty board
game_tree = iterate_tree(next_ttt_states)(init_state)

Let's check its first level, i.e., the valid moves for first player:

In [ ]:
prune(1)(game_tree)

.../.../... [X] 0
   X../.../... [O] 0
   .X./.../... [O] 0
   ..X/.../... [O] 0
   .../X../... [O] 0
   .../.X./... [O] 0
   .../..X/... [O] 0
   .../.../X.. [O] 0
   .../.../.X. [O] 0
   .../.../..X [O] 0

Another example, taking a position with some stones, check the ramaining game tree:

In [ ]:
state = TTT(list('XX.O.O.XO'), 'X')
game_tree = iterate_tree(next_ttt_states)(state)

end_game = prune(3)(game_tree)
end_game

XX./O.O/.XO [X] 0
   XXX/O.O/.XO [O] 0
   XX./OXO/.XO [O] 0
   XX./O.O/XXO [O] 0
      XXO/O.O/XXO [X] 0
      XX./OOO/XXO [X] 0

Right now, the value attribute for each state is still zero.

Function `eval_board` assigns values to end positions,

In [ ]:
def eval_board(board):
  if has_win(board, 'X'):
    return 1
  if has_win(board, 'O'):
    return -1
  return 0

> it would be possible to create elaborate heuristics to assign numbers between `[-1,1]` to help a computer player to navigate the game tree, but that's not our goal here.

The next function initializes the (possibly pruned) game tree with its winning positions,

In [ ]:
def eval_tree(f, tree):
  tree.state.value = eval_board(tree.state.board)
  for child in tree.children:
    eval_tree(f, child)

In [ ]:
eval_tree(eval_board, end_game)
end_game

XX./O.O/.XO [X] 0
   XXX/O.O/.XO [O] 1
   XX./OXO/.XO [O] 1
   XX./O.O/XXO [O] 0
      XXO/O.O/XXO [X] -1
      XX./OOO/XXO [X] -1

We can now apply min-max to the game tree, to decide, for every upper node, if there's a winning strategy,

In [ ]:
def max_tree_value(tree):
  if tree.children:
    tree.state.value = max(map(min_tree_value, tree.children))
  return tree.state.value

def min_tree_value(tree):
  if tree.children:
    tree.state.value = min(map(max_tree_value, tree.children))
  return tree.state.value

In [ ]:
max_tree_value(end_game)
end_game # if node is 1/-1, there's a winnable position for X/O

XX./O.O/.XO [X] 1
   XXX/O.O/.XO [O] 1
   XX./OXO/.XO [O] 1
   XX./O.O/XXO [O] -1
      XXO/O.O/XXO [X] -1
      XX./OOO/XXO [X] -1

A larger game tree example,

In [ ]:
state = TTT(list('X..O..X.O'), 'X')                # game: X..
game_tree = iterate_tree(next_ttt_states)(state)   #       O..
game2 = prune(5)(game_tree)                        #       X.O

eval_tree(eval_board, game2)
max_tree_value(game2)

game2 # it's possible to win if X plays at upper-right corner

X../O../X.O [X] 1
   XX./O../X.O [O] 0
      XXO/O../X.O [X] 0
         XXO/OX./X.O [O] -1
            XXO/OXO/X.O [X] -1
            XXO/OX./XOO [X] 0
               XXO/OXX/XOO [O] 0
         XXO/O.X/X.O [O] 0
            XXO/OOX/X.O [X] 0
               XXO/OOX/XXO [O] 0
            XXO/O.X/XOO [X] 0
               XXO/OXX/XOO [O] 0
         XXO/O../XXO [O] -1
            XXO/OO./XXO [X] 0
               XXO/OOX/XXO [O] 0
            XXO/O.O/XXO [X] -1
      XX./OO./X.O [X] 1
         XXX/OO./X.O [O] 1
         XX./OOX/X.O [O] 0
            XXO/OOX/X.O [X] 0
               XXO/OOX/XXO [O] 0
            XX./OOX/XOO [X] 1
               XXX/OOX/XOO [O] 1
         XX./OO./XXO [O] -1
            XXO/OO./XXO [X] 0
               XXO/OOX/XXO [O] 0
            XX./OOO/XXO [X] -1
      XX./O.O/X.O [X] 1
         XXX/O.O/X.O [O] 1
         XX./OXO/X.O [O] -1
            XXO/OXO/X.O [X] -1
            XX./OXO/XOO [X] 1
               XXX/OXO/XOO [O] 1
         XX./O.O/XXO [O] -1
            X

# Power Series

This section is based on chapters 8,10 of Doets and van Eijck's [The Haskell Road to Logic, Math & Programming](https://fldit-www.cs.tu-dortmund.de/~peter/PS07/HR.pdf) and McIlroy's [Power Series, Power Serious](https://www.cambridge.org/core/journals/journal-of-functional-programming/article/power-series-power-serious/19863F4EAACC33E1E01DE2A2114EC7DF).

In [ ]:
# @title Helper functions
from fractions import Fraction

# note: head() and show() consume the generator
def head(xs, n):
  """ generates the first n items """
  for i, x in enumerate(xs):
    if i == n:
      break
    yield maybe_frac(x)

def maybe_frac(x):
  if type(x) != Fraction:
    return x
  return x.numerator if x.denominator == 1 else f'{x.numerator}/{x.denominator}'

show = lambda xs, n=10: print(*head(xs, n))

Let's consider power series, which are algebraic objects like polynomials but having an infinite number of coefficients.

A power series $f$ in variable $z$ is given by expression

$$f(z) = f_0 + z f_1 + z^2f_2 + z^3f_3 \ldots$$

It is a useful notation to consider a power series consisting of a head $f_0$ and tail $\overline{f}$:

$$f = f_0 + z \overline{f}$$

where $\overline{f} = f_1 + zf_2 + z^2f_3 + \ldots$

The representation of a power series in Python is a generator of rational coefficients $f_0, f_1, f_2, \ldots$, where variable $z$ is implicit.

The next examples show generators for the series $0, 0, 0, \ldots$ and $1, 1, 1, \ldots$ respectively

In [ ]:
def zeros():
  while True:
    yield Fraction(0)

def ones():
  while True:
    yield Fraction(1)

Using this representation it's possible to define functions that manipulate and combine power series, like for eg,

In [ ]:
def negate(xs):
  for x in xs:
    yield -x

In [ ]:
show(ones())
show(negate(ones()))

1 1 1 1 1 1 1 1 1 1
-1 -1 -1 -1 -1 -1 -1 -1 -1 -1


It's also possible to transform polynomials into power series by adding an infinite sequence of zeros for the larger exponents:

In [ ]:
def series(*xs):
  yield from (Fraction(x) for x in xs)
  yield from zeros()

In [ ]:
show(series(1,0,2,3))

1 0 2 3 0 0 0 0 0 0


To add two power series:

$$f + g = (f_0 + z \overline{f}) + (g_0 + z \overline{g}) = (f_0+g_0) + z(\overline{f} + \overline{g})$$

we need to sum each pair of coefficients of the same order:

In [ ]:
# f and g must not be the same generator (the same applies to all binary operations)
def add(f, g):
  while True:
    yield next(f) + next(g)

In [ ]:
show(add(ones(), ones()))

2 2 2 2 2 2 2 2 2 2


Substraction is basically the same idea:

In [ ]:
def sub(f, g):
  while True:
    yield next(f) - next(g)

In [ ]:
show(sub(ones(), ones()))

0 0 0 0 0 0 0 0 0 0


Scalar multiplication by constant $c$ is made by multiplying each coefficient by $c$,

In [ ]:
def scalar_mult(c, f):
  for fi in f:
    yield c * fi

In [ ]:
show(scalar_mult(10, ones()))

10 10 10 10 10 10 10 10 10 10


Let's consider multiplication of two power series:

$$f \times g = (f_0 + z \overline{f}) \times (g_0 + z \overline{g}) = f_0g_0 + z(g_0\overline{f} + f_0\overline{g} + z \overline{f}\overline{g}) = f_0g_0 + z(f_0\overline{g} + \overline{f}g)$$

Since generator $g$ is used twice in the final expression, we need to duplicate it using `itertools.tee` to prevent it to be consumed too soon,

In [ ]:
from itertools import tee

def mul(f, g):
  g_, g = tee(g, 2)
  f0, g0 = next(f), next(g_)
  yield f0 * g0
  yield from add(scalar_mult(f0, g_), mul(f, g))

In [ ]:
from itertools import count

show(mul(count(), ones()), n=20)  # the first twenty triangular numbers

0 1 3 6 10 15 21 28 36 45 55 66 78 91 105 120 136 153 171 190


This multiplication is a convolution between the coefficients of the two power series. The i-th coefficient of $f\times g$ is given by the sum of $f_i g_{i-j}$ for all $j=0..i$.

In [ ]:
d6 = lambda: series(1,1,1,1,1,1) # distribution of the six values of a dice
show(mul(d6(), d6()), n=11)      # convolution of two of these distributions

1 2 3 4 5 6 5 4 3 2 1


For division, the quotient $h$ of the division of power series $f,g$ is given by,

$$f = h \times g$$

if we expand,

$$f_0 + z\overline{f} = (h_0 + z\overline{h}) \times (g_0 + z\overline{g}) = h_0g_0 + z(h_0\overline{g} + \overline{h}g)$$

solving for $h_0$ and $h$:

$$h_0 = f_0 / g_0$$

$$\overline{h} = (\overline{f} - h_0\overline{g}) / g$$

In Python:

In [ ]:
def div(f, g):
  g_, g = tee(g, 2)
  f0, g0 = next(f), next(g_) # again duplicate g since it is used twice
  if f0 == 0 and g0 == 0:
    yield from div(f, g_)
  if g0 == 0:
    raise ZeroDivisionError()
  h0 = f0 / g0
  yield h0
  yield from div(sub(f, scalar_mult(h0, g_)), g)

As an example, the expression $\frac{1}{1-z}$ is not a polynomial, but a power series where all coefficients are one:

In [ ]:
a, b = series(1), series(1,-1)
show(div(a,b))

1 1 1 1 1 1 1 1 1 1


this makes sense, since $1+z+z^2+z^3+\ldots$ is the sum of a [geometric series](https://en.wikipedia.org/wiki/Geometric_series) with ratio $z$, which totals $1 / (1-z)$.

Another example is $\frac{1}{(1-z)^2}$ that generates all natural numbers. This is because

$$\frac{1}{(1-z)^2} = \frac{d}{dz} \frac{1}{1-z} = \frac{d}{dz} (1+z+z^2+z^3+\ldots) = 1 +2z + 3z^2 + \ldots$$


In [ ]:
a, b = series(1), mul(series(1,-1), series(1,-1))
show(div(a,b)) # 1/(1-z)²

1 2 3 4 5 6 7 8 9 10


Let's divide $(x^2 + x^3)/(2 + x)$ which should produce $x^2/2 + x^3/4 - x^4/8 + x^5/16 - x^6/32 + \ldots$

In [ ]:
a, b = series(0,0,1,1), series(2,1)
show(div(a,b))

0 0 1/2 1/4 -1/8 1/16 -1/32 1/64 -1/128 1/256


The polynomial ratio is called the **generating function** of the series.

The sequence $a_{n+1} = 2a_n + 1, a_0=0$ can be described by the following generating function,

$$\frac{x}{(1-x)(1-2x)}$$

Let's check if we get the expected results:

In [ ]:
a, b, c = series(0,1), series(1,-1), series(1,-2)
show(div(a, mul(b,c)))

0 1 3 7 15 31 63 127 255 511


Another sequence $a_{n+1} = 2a_n + n, a_0=1$ has the generating function,

$$\frac{1-2x+2x^2}{(1-x)^2(1-2x)}$$

which starts with values $1, 2, 5, 12, 27, 58, 121, ...$

In [ ]:
a, b, c = series(1,-2,2), series(1,-2,1), series(1,-2)
show(div(a, mul(b,c)))

1 2 5 12 27 58 121 248 503 1014


## Differentiation and Integration

To apply differentiation, we know that

$$\frac{d}{dz} z^n = n z^{n-1}$$

which depends on the term of the coefficient, so we need to keep that information while computing the derivative of each coefficient:

In [ ]:
def derivation(zs):
  def deriv(ys, n):
    yield n * next(ys)
    yield from deriv(ys, n+1)
  next(zs) # remove constant
  yield from deriv(iter(zs), Fraction(1))

In [ ]:
a = series(1,2,3,4) # 1 + 2x + 3x² + 4x³ + 0...
show(derivation(a))

2 6 12 0 0 0 0 0 0 0


The definitive integral $\int_0^z f(t) dt$ is the reversed computation:

In [ ]:
def integration(xs):  # integration ∫_0^z f(t) dt
  def integ(ys, n):
    yield next(ys) / n
    yield from integ(ys, n+1)
  yield 0  # not enough information to determine the constant
  yield from integ(iter(xs), Fraction(1))

In [ ]:
a = series(2,6,12) # reverting the previous derivative

show(integration(a))

0 2 3 4 0 0 0 0 0 0


Let's check the use of differentiation and integration with some famous power series.

The exponential function is generated by the series,

$$e^z = \frac{1}{0!} + \frac{1}{1!} + \frac{1}{2!} + \frac{1}{3!} + \ldots$$

We know that $e^x$ satisfies $\frac{d}{dx} e^x = e^x$ which satisfies the next differential equation:

$$\frac{d}{dx} y = y$$

with initial condition $y(0)=1$

Integrating we get,

$$y = 1+ \int_0^x y(t) dt$$

In Python:

In [ ]:
def exps():
  yield Fraction(1)
  gen = integration(exps())
  next(gen) # remove the zero (TODO: can this code be improved?)
  yield from gen

show(exps())

1 1 1/2 1/6 1/24 1/120 1/720 1/5040 1/40320 1/362880


It is known that $\frac{1}{1-z} f(z)$ is a series with the incremental sums of the coefficients of $f(x)$, so we can use `exps` to approximate the value of constant $e$,

In [ ]:
def e_series():
  g = div(series(1), sub(series(1), series(0,1)))  # g = 1/(1-z)
  yield from mul(g, exps())

show(e_series())

# print first approximations
print(*head((round(float(approx),6) for approx in e_series()), 10))

1 2 5/2 8/3 65/24 163/60 1957/720 685/252 109601/40320 98641/36288
1.0 2.0 2.5 2.666667 2.708333 2.716667 2.718056 2.718254 2.718279 2.718282


Let's consider $\sin$ and $\cos$ which expansions are:

$$\sin x = x - \frac{x^3}{3!} + \frac{x^5}{5!} - \frac{x^7}{7!} + \ldots$$

$$\cos x = 1 - \frac{x^2}{2!} + \frac{x^4}{4!} - \frac{x^6}{6!} + \ldots$$

we know that

$$\frac{d}{dx} \sin x = \cos x$$

$$\frac{d}{dx} \cos x = -\sin x$$

with $\sin(0)=0$ and $\cos(0)=1$.

In [ ]:
def sinx():
  yield 0
  gen = integration(cosx())
  next(gen) # remove the zero
  yield from gen

def cosx():
  yield 1
  gen = integration(negate(sinx()))
  next(gen) # remove the zero
  yield from gen

And by the marvel of mutual recursion, the power series are correctly computed:

In [ ]:
show(sinx())
show(cosx())

0 1 0 -1/6 0 1/120 0 -1/5040 0 1/362880
1 0 -1/2 0 1/24 0 -1/720 0 1/40320 0


## A type for Power Series


The next class joins the previous operations, using dunders to make it easier to combine power series.

In [ ]:
from types import GeneratorType
from itertools import islice, chain

class Series:
  def __init__(self, *coefs):
    if type(coefs[0]) in [GeneratorType, chain]:
      self.cs = coefs[0]
    else: # received finite #values, so make a series from them
      self.cs = series(*coefs)

  def __add__(self, p):
    return Series(add(iter(self), iter(p)))

  def __radd__(self, c):
    xs = iter(self)
    return Series(chain([c+next(xs)], xs))

  def __sub__(self, p):
    return Series(sub(iter(self), iter(p)))

  def __rsub__(self, c):
    xs = iter(self)
    return Series(chain([c-next(xs)], negate(xs)))

  def __mul__(self, p):
    return Series(mul(iter(self), iter(p)))

  def __rmul__(self, c):
    return Series(scalar_mult(c, iter(self)))

  def __pow__(self, n):
    if n == 0:
      return Series(Fraction(1))
    half = self ** (n//2)
    p = mul(iter(half), iter(half))
    if n%2 == 0:
      return Series(p)
    return Series(mul(p, iter(self)))

  def __truediv__(self, p):
    return Series(div(iter(self), iter(p)))

  def intr(self):
    return Series(integration(iter(self)))

  def deriv(self):
    return Series(derivation(iter(self)))

  def __iter__(self):
    """ return an iterator for processing, without draining the object generator """
    xs, self.cs = tee(self.cs, 2)
    return xs

  def __getitem__(self, index):
    for i, x in enumerate(iter(self)):
      if i == index:
        return x

  def __repr__(self):
    raise NotImplementedError('A power series cannot be fully represented')

Here's the generation of the naturals and the odd numbers:

In [ ]:
z = Series(0,1)

nats = Series(1) / (1 - z)**2
show(nats)

odds = Series(ones()) + 2*z*nats
show(odds)

1 2 3 4 5 6 7 8 9 10
1 3 5 7 9 11 13 15 17 19


Let's produce the first lines of the Pascal triangle (using the [binomial theorem](https://en.wikipedia.org/wiki/Binomial_theorem)):

In [ ]:
z1 = Series(1,1)

for n in range(10):
  show(z1**n, n+1)  # the coefficients of (x+1)^n

1
1 1
1 2 1
1 3 3 1
1 4 6 4 1
1 5 10 10 5 1
1 6 15 20 15 6 1
1 7 21 35 35 21 7 1
1 8 28 56 70 56 28 8 1
1 9 36 84 126 126 84 36 9 1


The distribution of the sums of three dices:

In [ ]:
d6 = Series(1,1,1,1,1,1)
show(d6*d6*d6, 16)

1 3 6 10 15 21 25 27 27 25 21 15 10 6 3 1


How to generate the power series $0,1,0,1,0,1,...$?

The sequence is recursively specified as $f(z) = 0 + z + z^2 f(z)$

$$f(z) = 0 + z + z^2 f(z) \iff f(z) - z^2 f(z) = z \iff f(z) = \frac{z}{1-z^2}$$

Let's try in Python:

In [ ]:
z = Series(0,1)
show(z / (1 - z**2))

0 1 0 1 0 1 0 1 0 1


To compute a power series with the powers of two as coefficients, we define the sequence recursively as $f(z) = 1 + 2zf(z)$ which means we double the known sequence and shift the exponents one value up (by multiplying with $z$),

$$f(z) = 1 + 2zf(z) \iff f(z) - 2zf(z) = 1 \iff f(z) = \frac{1}{1-2z}$$

In [ ]:
z = Series(0,1)
show( Series(1) / (1 - 2*z) )

1 2 4 8 16 32 64 128 256 512


We can use a similar reasoning to generate all inverses of the powers of two:

In [ ]:
z = Series(0,1)
show(Series(1) / (1 - Fraction(1,2)*z))  # f(z) = 1 + zf(z)/2

1 1/2 1/4 1/8 1/16 1/32 1/64 1/128 1/256 1/512


Or a series with values $1/3^n$,

In [ ]:
a, b = Series(3), Series(3,-1)  # f(z) = 1 + zf(z)/3 <=> f(z) = 3/(3-z)
show(a/b)

1 1/3 1/9 1/27 1/81 1/243 1/729 1/2187 1/6561 1/19683


A series with the inverses of the naturals can be made via integration:

In [ ]:
a = Series(1) / Series(1,-1) # ∫_0^z f(t) dt = ∫ 1 / (1-t) dt
show(a.intr())

0 1 1/2 1/3 1/4 1/5 1/6 1/7 1/8 1/9


There's a theorem stating that $z \frac{d}{dz} f(z)$ generates the series $0 f_0, 1f_1, 2f_2, 3f_3 \ldots$.

So, the squares can be made by differentiating the natural numbers times $z$:

In [ ]:
nats0 = z / (1 - z)**2
squares = z * nats0.deriv()
show(squares)

0 1 4 9 16 25 36 49 64 81


The next generator outputs the [Catalan numbers](https://en.wikipedia.org/wiki/Catalan_number),

In [ ]:
def catalan():
  yield 1
  yield from mul(catalan(), catalan())

In [ ]:
show(catalan())

1 1 2 5 14 42 132 429 1430 4862


One way to interpret the n-th Catalan number is the number of binary trees that is possible to have with $n$ nodes. Notice that if a tree has $n+1$ nodes, and  the left subtree has $i$ nodes, then the right subtree must have $n-i$ nodes, which is a convolutional operation.

The next example computes the power series with coefficients $(n+1)^2$ by summing three power series, namely $n^2, 2n, 1$,

In [ ]:
s_ones = Series(ones())
nplus1 = squares + 2*nats0 + s_ones
show(nplus1)

1 4 9 16 25 36 49 64 81 100


There's a theorem stating that $\frac{1}{1-z} f(z)$ generates the series $f_0, f_0+f_1, f_0+f_1+f_2, \ldots$.

There's also a theorem that states that $(n+1)^2$ is equal to the first $n$ odd numbers.

These two theorems provide another solution to the previous problem:

In [ ]:
nplus1 = odds / (1-z)
show(nplus1)

1 4 9 16 25 36 49 64 81 100


When we defined the series $f$ for the squares, we evaluated

$$f(z) = \frac{d}{dz} \frac{z}{(1-z)^2} = \frac{1}{(1-z)^2} + \frac{2z}{(1-z)^3}$$

and since $\frac{1}{1-z}$ is the series of all ones, we can also define the $(n+1)^2$ series as,

In [ ]:
nplus1 = s_ones**2 + 2*z * s_ones**3
show(nplus1)

1 4 9 16 25 36 49 64 81 100


The next series represents the Fibonnaci sequence (starting with 0,1,...),

$$f(z) = 0 + z + z^2\Big(f(z) + \frac{f(z)}{z}\Big)$$

since dividing by $z$ shifts left (as multiplying by $z$ shifts right).

So,

$$f(z) = z +z^2f(z) + zf(z) \iff f(z) = \frac{z}{1-z-z^2}$$

In [ ]:
fibs = z / (1 - z - z**2)
show(fibs, 30)

0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 6765 10946 17711 28657 46368 75025 121393 196418 317811 514229


Notice how fast is the computation of the values in Python, because the generators are not duplicated but shared (due to the application of the `tee` function).

The general expression for the series of a Fibonacci-like sequence starting as $f_0,f_1,...$ is

$$f(z) = \frac{f_0(1-z) + f_1z}{1-z-z^2}$$

because: $z^n f_{n+1} = z^n f_n + z^n f_{n-1}$ for $n \geq1$

$$z^n f_{n+1} = zf_2 + z^2f_3 + z^3f_4 + \ldots = \frac{f(z) - f_0 - zf_1}{z}$$

and

$$z^n f_n + z^n f_{n-1} = (zf_1 + z^2f_2 + \ldots) + (zf_0 + z^2f_1 + \ldots) = (f(z)-f_0) + zf(z) = (1+z)f(z) - f_0$$

combining both sides (and multiplying them by $z$),

$$f(z) - f_0 - zf_1 = (z+z^2)f(z) - zf_0$$

which results in the first expression.

<!-- ref: https://ektalks.blogspot.com/2018/07/derivation-of-nth-term-of-lucas.html -->

As an example, the next series produces the Lucas numbers (the same as Fibonacci, but starting with 2,1,...),

In [ ]:
lucas = (2*(1-z) + z) / (1 - z - z**2)
show(lucas, 20)

2 1 3 4 7 11 18 29 47 76 123 199 322 521 843 1364 2207 3571 5778 9349


## Combinatorial Problems

We can use power series to solve combinatorial problems. Some examples:

> _How many ways are there of selecting ten red, blue or white marbles
from a vase, in such a way that there are at least two of each color and at most
five marbles have the same colour?_

This can be done by evaluating the coefficient of $z^{10}$ of the following polynomial,

$$(z^2+z^3+z^4+z^5)^3$$

In [ ]:
z = Series(0,1)

sol = (z**2 + z**3 + z**4 + z**5)**3

print(sol[10])

12


> _There are 3 pieces of 1 gram weight, 4 pieces of 2 grams weight, 2 pieces of 4 grams weight, how many different ways to weigh a total of 6 grams?_

In [ ]:
z = Series(0,1)

pieces_1g = 1 + z + z**2 + z**3           # up to 3 pieces of 1 gram
pieces_2g = 1 + z**2 + z**4 + z**6 + z**8 # up to 4 pieces of 2 gram
pieces_4g = 1 + z**4 + z**8               # up to 2 pieces of 4 gram

# produce the power series (aka, the generating function) of the problem
prob = pieces_1g * pieces_2g * pieces_4g

show(prob, 20) # the entire solution of each total of N grams

1 1 2 2 3 3 4 4 5 5 5 5 4 4 3 3 2 2 1 1


In this case, we want to know the value for 6 grams, so the coefficient $z^6$,

In [ ]:
prob[6]

Fraction(4, 1)

Indeed there are four solutions: $4+1+1; 2+4; 2+2+1+1; 2+2+2$.

> _We have four types of coins, 5 cents, 10 cents, 20 cents and 50 cents. What are the possible combinations for amounts up to 100 cents?_

In [ ]:
# exponentiation is too costly, so let's explicitly define each power series
coin05 = Series(*[int(i%5 ==0) for i in range(0, 101)])
coin10 = Series(*[int(i%10==0) for i in range(0, 101)])
coin20 = Series(*[int(i%20==0) for i in range(0, 101)])
coin50 = Series(*[int(i%50==0) for i in range(0, 101)])

prob = coin05*coin10*coin20*coin50

total = sum(int(prob[i]) for i in range(0,101))
total

343

In the previous problems the order of the elements is not relevant.

To deal with combinatorial problems where order matters, we can use what's denoted as **exponential generating functions** where each coefficient $f_i$ is divided by its factorial $i!$,

$$f(z) = f_0 + f_1z + \frac{f_2 z^2}{2!} + \frac{f_3 z^3}{3!} + \ldots$$

The next function converts an ordinary generating function to its respective exponential generating function,

In [ ]:
def o2e(f):
  f_, f = tee(f, 2)
  yield next(f_)
  yield from o2e(derivation(f))

For instance, converting the exponential series to its exponential variant, gives us $\frac{1}{1-z}$ (i.e., the series with all ones),

In [ ]:
show(exps())
show(o2e(exps()))

1 1 1/2 1/6 1/24 1/120 1/720 1/5040 1/40320 1/362880
1 1 1 1 1 1 1 1 1 1


Doing it again gives us the factorials:

In [ ]:
show(o2e(o2e(exps())))

1 1 2 6 24 120 720 5040 40320 362880


The next converts an exponential series to its ordinary form:

In [ ]:
def e2o(f):
  yield next(f)
  gen = integration(e2o(f))
  next(gen)
  yield from gen

For instance, converting the exponential series $\frac{1}{1-z}$ (i.e., the series with all ones) to its ordinary variant, gives us the coefficients of $e^z$,

In [ ]:
show(e2o(ones()))

1 1 1/2 1/6 1/24 1/120 1/720 1/5040 1/40320 1/362880


For example:

+ Series $(1+z)^r$ in its standard form represents the problem of picking a subset of size $n$ from a set of size $r$ (i.e., combinations)

+ But series $(1+z)^r$ in its exponential form represents the problem of picking a sequence of $n$ distinct objects from a set of size $r$ (i.e., permutations)

In [ ]:
r = 10
s = (1+z)**r

show(s, 11)  # combinations of i elements over r elements
show(o2e(s)) # permutations of i elements over r elements

1 10 45 120 210 252 210 120 45 10 1
1 10 90 720 5040 30240 151200 604800 1814400 3628800


> _Suppose a vase contains red, white and blue marbles, at least four
of each kind. How many ways are there of arranging sequences of marbles from the vase, with at most four marbles of the same colour?_

The series for one marble must be $1 + z + \frac{z^2}{2!} + \frac{z^3}{3!} + \frac{z^4}{4!}$ since, for $n$ marbles of the same color, divide by $n!$, i.e., the number of permutations that are indistinguible:

In [ ]:
show(e2o(series(1,1,1,1,1)))

1 1 1/2 1/6 1/24 0 0 0 0 0


Since there are three marble types (all with the same restrictions), we cube the series:

In [ ]:
marble = e2o(series(1,1,1,1,1))
sol = Series(marble)**3

sol = o2e(sol.cs) # get object's generator and convert to exponential form to count permutations
show(sol)         # sol[i] is the answer for a total of 'i' marbles

1 3 9 27 81 240 690 1890 4830 11130


> _Suppose a vase contains red, white and blue marbles, an unlimited
supply of each colour. How many ways are there of arranging sequences of
marbles from the vase, assume that at least two marbles are of the same colour?_

The series is similar to the previous $1 + z + \frac{z^2}{2!} + \frac{z^3}{3!} + \frac{z^4}{4!}$, but now without limit and we need to remove the options for zero or one marbles. This results in $e^z - 1 - z$

In [ ]:
show(Series(exps()) - (1+z))  # e^z - 1 - z

0 0 1/2 1/6 1/24 1/120 1/720 1/5040 1/40320 1/362880


Again, since there are three marble types, we cube the series

In [ ]:
sol = (Series(exps()) - (1+z))**3
sol = o2e(sol.cs) # convert to exponential form to count permutations

show(sol,15)      # sol[6] is the first satisfying two marbles of each type (90 = 6!/2³)

0 0 0 0 0 0 90 630 2940 11508 40950 137610 445896 1410552 4390386


> _Suppose a vase contains red, white and blue marbles, at least four
of each kind. How many ways are there of arranging sequences of marbles from
the vase, with at least two and at most four marbles of the same colour?_

In [ ]:
show(e2o(series(0,0,1,1,1))) # possible configurations for one marble

0 0 1/2 1/6 1/24 0 0 0 0 0


In [ ]:
sol = Series(e2o(series(0,0,1,1,1)))**3
sol = o2e(sol.cs) # convert to exponential form to count permutations

show(sol,15)      # sol[6] is the first satisfying two marbles of each type
                  # while s[12] is the last one (more than 12 marbles is impossible)

0 0 0 0 0 0 90 630 2940 9240 22050 34650 34650 0 0


# References

+ Turner - [Total Functional Programming](http://sblp2004.ic.uff.br/papers/turner.pdf) (2004)

+ Mike Gordon - [Corecursion and Coinduction](https://www.semanticscholar.org/paper/Corecursion-and-coinduction-%3A-what-they-are-and-how-Gordon/41fb876f6b35971173ef1808472350b51cf3afd1) (2007)

+ John Hughes - [Why Functional Programming Matters](https://www.cs.kent.ac.uk/people/staff/dat/miranda/whyfp90.pdf) (1990)

+ Benjamin Jones - [Alpha-Beta Pruning](https://web.archive.org/web/20180819083507/http://bfj7.com/blog/alpha-beta-pruning.html) (2013)

+ Doets and van Eijck's [The Haskell Road to Logic, Math & Programming](https://fldit-www.cs.tu-dortmund.de/~peter/PS07/HR.pdf) (2004)

+ McIlroy's [Power Series, Power Serious](https://www.cambridge.org/core/journals/journal-of-functional-programming/article/power-series-power-serious/19863F4EAACC33E1E01DE2A2114EC7DF) (1999)

+ Wikipedia, [Corecursion](https://en.wikipedia.org/wiki/Corecursion)

